In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

/home/shimazu/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def n(digits=3):
    number = ''
    for i in range(np.random.randint(1, digits + 1)):
        number += np.random.choice(list('0123456789'))
    return int(number)

In [3]:
def padding(chars, maxlen):
    return chars + ' ' * (maxlen - len(chars))

In [4]:
N = 20000
N_train = int(N * 0.9)
N_validation = N - N_train

digits = 3
input_digits = digits * 2 + 1
output_digits = digits + 1

added = set()
questions = []
answers = []

while len(questions) < N:
    a, b = n(), n()
    
    pair = tuple(sorted((a, b)))
    if pair in added:
        continue
        
    question = '{}+{}'.format(a, b)
    question = padding(question, input_digits)
    answer = str(a + b)
    answer = padding(answer, output_digits)
    
    added.add(pair)
    questions.append(question)
    answers.append(answer)

In [5]:
chars = '0123456789+ '
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [6]:
X = np.zeros((len(questions), input_digits, len(chars)), dtype=np.integer)
Y = np.zeros((len(questions), digits + 1, len(chars)), dtype=np.integer)

for i in range(N):
    for t, char in enumerate(questions[i]):
        X[i, t, char_indices[char]] = 1
    for t, char in enumerate(answers[i]):
        Y[i, t, char_indices[char]] = 1

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, train_size=N_train)

/home/shimazu/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [7]:
def inference(x, y, n_batch, is_training,
              input_digits=None, output_digits=None,
              n_hidden=None, n_out=None):
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(initial)

    def bias_variable(shape):
        initial = tf.zeros(shape, dtype=tf.float32)
        return tf.Variable(initial)

    # Encoder
    encoder = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    state = encoder.zero_state(n_batch, tf.float32)
    encoder_outputs = []
    encoder_states = []

    with tf.variable_scope('Encoder'):
        for t in range(input_digits):
            if t > 0:
                tf.get_variable_scope().reuse_variables()
            (output, state) = encoder(x[:, t, :], state)
            encoder_outputs.append(output)
            encoder_states.append(state)

    # Decoder
    decoder = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    state = encoder_states[-1]
    decoder_outputs = [encoder_outputs[-1]]

    # 出力層の重みとバイアスを事前に定義
    V = weight_variable([n_hidden, n_out])
    c = bias_variable([n_out])
    outputs = []

    with tf.variable_scope('Decoder'):
        for t in range(1, output_digits):
            if t > 1:
                tf.get_variable_scope().reuse_variables()

            if is_training is True:
                (output, state) = decoder(y[:, t-1, :], state)
            else:
                # 直前の出力を入力に用いる
                linear = tf.matmul(decoder_outputs[-1], V) + c
                out = tf.nn.softmax(linear)
                outputs.append(out)
                out = tf.one_hot(tf.argmax(out, -1), depth=output_digits)
                (output, state) = decoder(out, state)

            decoder_outputs.append(output)

    if is_training is True:
        output = tf.reshape(tf.concat(decoder_outputs, axis=1),
                            [-1, output_digits, n_hidden])

        linear = tf.einsum('ijk,kl->ijl', output, V) + c
        # linear = tf.matmul(output, V) + c
        return tf.nn.softmax(linear)
    else:
        # 最後の出力を求める
        linear = tf.matmul(decoder_outputs[-1], V) + c
        out = tf.nn.softmax(linear)
        outputs.append(out)

        output = tf.reshape(tf.concat(outputs, axis=1),
                            [-1, output_digits, n_out])
        return output

In [8]:
def loss(y, t):
    return tf.reduce_mean(tf.square(y-t))

def training(loss):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999)
    
    train_step = optimizer.minimize(loss)
    return train_step

In [9]:
def accuracy(y, t):
    correct_prediction = tf.equal(tf.argmax(y, -1), tf.argmax(t, -1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [10]:
n_in = len(chars)
n_hidden = 128
n_out = len(chars)

x = tf.placeholder(tf.float32, shape=[None, input_digits, n_in])
t = tf.placeholder(tf.float32, shape=[None, output_digits, n_out])
n_batch = tf.placeholder(tf.int32, shape=[])
is_training = tf.placeholder(tf.bool)

y = inference(x, t, n_batch, is_training,
                  input_digits=input_digits,
                  output_digits=output_digits,
                  n_hidden=n_hidden, n_out=n_out)
loss = loss(y, t)
train_step = training(loss)

acc = accuracy(y, t)
history = {
        'val_loss': [],
        'val_acc': []
    }

In [11]:
epochs = 200
batch_size = 200

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

n_batches = N_train // batch_size

In [12]:
for epoch in range(epochs):
    print('=' * 10)
    print('Epoch:', epoch)
    print('=' * 10)

    X_, Y_ = shuffle(X_train, Y_train)

    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size

        sess.run(train_step, feed_dict={
            x: X_[start:end],
            t: Y_[start:end],
            n_batch: batch_size,
            is_training: True
        })

    # 検証データを用いた評価
    val_loss = loss.eval(session=sess, feed_dict={
        x: X_validation,
        t: Y_validation,
        n_batch: N_validation,
        is_training: False
    })
    val_acc = acc.eval(session=sess, feed_dict={
        x: X_validation,
        t: Y_validation,
        n_batch: N_validation,
        is_training: False
    })

    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    print('validation loss:', val_loss)
    print('validation acc: ', val_acc)

Epoch: 0
validation loss: 0.0593419
validation acc:  0.3525
Epoch: 1
validation loss: 0.0581544
validation acc:  0.352125
Epoch: 2
validation loss: 0.0578293
validation acc:  0.3535
Epoch: 3
validation loss: 0.057663
validation acc:  0.3575
Epoch: 4
validation loss: 0.0574567
validation acc:  0.358375
Epoch: 5
validation loss: 0.0571819
validation acc:  0.367375
Epoch: 6
validation loss: 0.0563238
validation acc:  0.374625
Epoch: 7
validation loss: 0.0563065
validation acc:  0.3775
Epoch: 8
validation loss: 0.0544865
validation acc:  0.396375
Epoch: 9
validation loss: 0.0532548
validation acc:  0.41225
Epoch: 10
validation loss: 0.0515745
validation acc:  0.440375
Epoch: 11
validation loss: 0.0501383
validation acc:  0.457875
Epoch: 12
validation loss: 0.0482513
validation acc:  0.47475
Epoch: 13
validation loss: 0.0470425
validation acc:  0.4855
Epoch: 14
validation loss: 0.0459659
validation acc:  0.499625
Epoch: 15
validation loss: 0.0450718
validation acc:  0.510375
Epoch: 16
valid

validation loss: 0.0267438
validation acc:  0.75075
Epoch: 99
validation loss: 0.026936
validation acc:  0.748125
Epoch: 100
validation loss: 0.0267858
validation acc:  0.75075
Epoch: 101
validation loss: 0.0269325
validation acc:  0.752125
Epoch: 102
validation loss: 0.0266614
validation acc:  0.7545
Epoch: 103
validation loss: 0.0265066
validation acc:  0.751875
Epoch: 104
validation loss: 0.0266199
validation acc:  0.756125
Epoch: 105
validation loss: 0.0262071
validation acc:  0.759125
Epoch: 106
validation loss: 0.0261886
validation acc:  0.756875
Epoch: 107
validation loss: 0.0261064
validation acc:  0.76225
Epoch: 108
validation loss: 0.0261934
validation acc:  0.763875
Epoch: 109
validation loss: 0.0261756
validation acc:  0.763125
Epoch: 110
validation loss: 0.0262239
validation acc:  0.764875
Epoch: 111
validation loss: 0.0258122
validation acc:  0.76875
Epoch: 112
validation loss: 0.0255771
validation acc:  0.769875
Epoch: 113
validation loss: 0.0257334
validation acc:  0.77

validation loss: 0.026625
validation acc:  0.789
Epoch: 196
validation loss: 0.0258729
validation acc:  0.794375
Epoch: 197
validation loss: 0.0255101
validation acc:  0.79775
Epoch: 198
validation loss: 0.0251527
validation acc:  0.80025
Epoch: 199
validation loss: 0.0250423
validation acc:  0.802625


In [13]:
# 検証データからランダムに問題を選んで答え合わせ
for i in range(10):
    index = np.random.randint(0, N_validation)
    question = X_validation[np.array([index])]
    answer = Y_validation[np.array([index])]
    prediction = y.eval(session=sess, feed_dict={
        x: question,
        # t: answer,
        n_batch: 1,
        is_training: False
    })
    question = question.argmax(axis=-1)
    answer = answer.argmax(axis=-1)
    prediction = np.argmax(prediction, -1)

    q = ''.join(indices_char[i] for i in question[0])
    a = ''.join(indices_char[i] for i in answer[0])
    p = ''.join(indices_char[i] for i in prediction[0])

    print('-' * 10)
    print('Q:  ', q)
    print('A:  ', p)
    print('T/F:', end=' ')
    if a == p:
        print('T')
    else:
        print('F')
print('-' * 10)

----------
Q:   549+975
A:   1521
T/F: F
----------
Q:   40+26  
A:   66  
T/F: T
----------
Q:   66+413 
A:   479 
T/F: T
----------
Q:   46+1   
A:   47  
T/F: T
----------
Q:   530+1  
A:   531 
T/F: T
----------
Q:   92+46  
A:   138 
T/F: T
----------
Q:   0+66   
A:   66  
T/F: T
----------
Q:   0+868  
A:   868 
T/F: T
----------
Q:   916+364
A:   1301
T/F: F
----------
Q:   302+5  
A:   306 
T/F: F
----------


# Kerasによる実装

In [15]:
from keras.optimizers import Adam
from keras.layers import Activation, Dense
from keras.layers.core import RepeatVector
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.models import Sequential

In [16]:
model = Sequential()

# Encoder
model.add(LSTM(n_hidden, input_shape=(input_digits, n_in)))

# Decoder
model.add(RepeatVector(output_digits))
model.add(LSTM(n_hidden, return_sequences=True))
model.add(TimeDistributed(Dense(n_out)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
             metrics=['accuracy'])

In [ ]:
epochs = 200
batch_size = 200

for epoch in range(epochs):
    model.fit(X_train, Y_train, batch_size=batch_size, epochs=1,
              validation_data=(X_validation, Y_validation))

    # 検証データからランダムに問題を選んで答え合わせ
    for i in range(10):
        index = np.random.randint(0, N_validation)
        question = X_validation[np.array([index])]
        answer = Y_validation[np.array([index])]
        prediction = model.predict_classes(question, verbose=0)

        question = question.argmax(axis=-1)
        answer = answer.argmax(axis=-1)

        q = ''.join(indices_char[i] for i in question[0])
        a = ''.join(indices_char[i] for i in answer[0])
        p = ''.join(indices_char[i] for i in prediction[0])

        print('-' * 10)
        print('Q:  ', q)
        print('A:  ', p)
        print('T/F:', end=' ')
        if a == p:
            print('T')
        else:
            print('F')
    print('-' * 10)

Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 7s 363us/step - loss: 2.0353 - acc: 0.2884 - val_loss: 1.8333 - val_acc: 0.3511
----------
Q:   54+641 
A:   110 
T/F: F
----------
Q:   81+823 
A:   110 
T/F: F
----------
Q:   21+654 
A:   119 
T/F: F
----------
Q:   362+28 
A:   110 
T/F: F
----------
Q:   6+181  
A:   11  
T/F: F
----------
Q:   23+869 
A:   110 
T/F: F
----------
Q:   73+760 
A:   110 
T/F: F
----------
Q:   2+485  
A:   119 
T/F: F
----------
Q:   95+546 
A:   110 
T/F: F
----------
Q:   312+6  
A:   11  
T/F: F
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 329us/step - loss: 1.7957 - acc: 0.3503 - val_loss: 1.7769 - val_acc: 0.3570
----------
Q:   578+70 
A:   119 
T/F: F
----------
Q:   751+6  
A:   12  
T/F: F
----------
Q:   123+829
A:   119 
T/F: F
----------
Q:   53+526 
A:   139 
T/F: F
----------
Q:   853+667
A:   110 
T/F: F
--

18000/18000 [==============================] - 6s 329us/step - loss: 1.3808 - acc: 0.4891 - val_loss: 1.3716 - val_acc: 0.4885
----------
Q:   373+0  
A:   377 
T/F: F
----------
Q:   94+46  
A:   143 
T/F: F
----------
Q:   415+3  
A:   458 
T/F: F
----------
Q:   77+808 
A:   888 
T/F: F
----------
Q:   23+24  
A:   33  
T/F: F
----------
Q:   348+4  
A:   448 
T/F: F
----------
Q:   5+899  
A:   986 
T/F: F
----------
Q:   37+208 
A:   380 
T/F: F
----------
Q:   120+222
A:   254 
T/F: F
----------
Q:   659+635
A:   1364
T/F: F
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 333us/step - loss: 1.3285 - acc: 0.5096 - val_loss: 1.3258 - val_acc: 0.5095
----------
Q:   72+942 
A:   102 
T/F: F
----------
Q:   200+210
A:   210 
T/F: F
----------
Q:   903+71 
A:   901 
T/F: F
----------
Q:   853+667
A:   1510
T/F: F
----------
Q:   364+5  
A:   477 
T/F: F
----------
Q:   543+8  
A:   552 
T/F: F
----------
Q:   494+

18000/18000 [==============================] - 6s 326us/step - loss: 0.7919 - acc: 0.7051 - val_loss: 0.8124 - val_acc: 0.6830
----------
Q:   48+93  
A:   132 
T/F: F
----------
Q:   795+50 
A:   800 
T/F: F
----------
Q:   557+988
A:   1544
T/F: F
----------
Q:   58+120 
A:   179 
T/F: F
----------
Q:   4+107  
A:   110 
T/F: F
----------
Q:   103+7  
A:   108 
T/F: F
----------
Q:   475+10 
A:   477 
T/F: F
----------
Q:   82+105 
A:   214 
T/F: F
----------
Q:   107+63 
A:   171 
T/F: F
----------
Q:   291+587
A:   978 
T/F: F
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 328us/step - loss: 0.7512 - acc: 0.7209 - val_loss: 0.7753 - val_acc: 0.6991
----------
Q:   59+615 
A:   672 
T/F: F
----------
Q:   39+49  
A:   97  
T/F: F
----------
Q:   261+32 
A:   273 
T/F: F
----------
Q:   479+471
A:   901 
T/F: F
----------
Q:   231+0  
A:   222 
T/F: F
----------
Q:   227+1  
A:   228 
T/F: T
----------
Q:   360+

18000/18000 [==============================] - 6s 328us/step - loss: 0.3422 - acc: 0.9012 - val_loss: 0.4039 - val_acc: 0.8619
----------
Q:   4+539  
A:   543 
T/F: T
----------
Q:   9+608  
A:   617 
T/F: T
----------
Q:   69+15  
A:   84  
T/F: T
----------
Q:   231+0  
A:   231 
T/F: T
----------
Q:   45+51  
A:   96  
T/F: T
----------
Q:   742+580
A:   1382
T/F: F
----------
Q:   518+388
A:   857 
T/F: F
----------
Q:   327+603
A:   959 
T/F: F
----------
Q:   17+24  
A:   41  
T/F: T
----------
Q:   404+8  
A:   412 
T/F: T
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 320us/step - loss: 0.3244 - acc: 0.9074 - val_loss: 0.3851 - val_acc: 0.8721
----------
Q:   445+69 
A:   514 
T/F: T
----------
Q:   31+876 
A:   907 
T/F: T
----------
Q:   88+667 
A:   765 
T/F: F
----------
Q:   506+1  
A:   507 
T/F: T
----------
Q:   69+15  
A:   84  
T/F: T
----------
Q:   55+26  
A:   81  
T/F: T
----------
Q:   952+

18000/18000 [==============================] - 6s 327us/step - loss: 0.1594 - acc: 0.9613 - val_loss: 0.2564 - val_acc: 0.9130
----------
Q:   553+552
A:   1105
T/F: T
----------
Q:   205+65 
A:   270 
T/F: T
----------
Q:   45+314 
A:   359 
T/F: T
----------
Q:   181+73 
A:   254 
T/F: T
----------
Q:   8+688  
A:   696 
T/F: T
----------
Q:   51+741 
A:   792 
T/F: T
----------
Q:   628+9  
A:   637 
T/F: T
----------
Q:   611+10 
A:   621 
T/F: T
----------
Q:   881+466
A:   1357
T/F: F
----------
Q:   453+4  
A:   457 
T/F: T
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 326us/step - loss: 0.1495 - acc: 0.9651 - val_loss: 0.2368 - val_acc: 0.9192
----------
Q:   795+709
A:   1594
T/F: F
----------
Q:   127+86 
A:   213 
T/F: T
----------
Q:   617+55 
A:   672 
T/F: T
----------
Q:   863+561
A:   1413
T/F: F
----------
Q:   735+289
A:   1034
T/F: F
----------
Q:   171+82 
A:   253 
T/F: T
----------
Q:   739+

...

18000/18000 [==============================] - 6s 325us/step - loss: 0.0198 - acc: 0.9994 - val_loss: 0.1365 - val_acc: 0.9526
----------
Q:   22+12  
A:   34  
T/F: T
----------
Q:   387+350
A:   737 
T/F: T
----------
Q:   40+221 
A:   261 
T/F: T
----------
Q:   400+12 
A:   422 
T/F: F
----------
Q:   371+10 
A:   381 
T/F: T
----------
Q:   670+132
A:   802 
T/F: T
----------
Q:   750+962
A:   1702
T/F: F
----------
Q:   198+57 
A:   255 
T/F: T
----------
Q:   548+3  
A:   551 
T/F: T
----------
Q:   578+70 
A:   648 
T/F: T
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 323us/step - loss: 0.0184 - acc: 0.9997 - val_loss: 0.1417 - val_acc: 0.9519
----------
Q:   253+97 
A:   340 
T/F: F
----------
Q:   547+0  
A:   547 
T/F: T
----------
Q:   16+37  
A:   53  
T/F: T
----------
Q:   194+77 
A:   271 
T/F: T
----------
Q:   4+274  
A:   278 
T/F: T
----------
Q:   891+259
A:   1230
T/F: F
----------
Q:   932+

18000/18000 [==============================] - 6s 327us/step - loss: 0.0115 - acc: 0.9999 - val_loss: 0.1288 - val_acc: 0.9577
----------
Q:   15+430 
A:   445 
T/F: T
----------
Q:   137+39 
A:   176 
T/F: T
----------
Q:   25+312 
A:   337 
T/F: T
----------
Q:   29+581 
A:   610 
T/F: T
----------
Q:   47+93  
A:   140 
T/F: T
----------
Q:   800+614
A:   1414
T/F: T
----------
Q:   1+335  
A:   336 
T/F: T
----------
Q:   13+277 
A:   290 
T/F: T
----------
Q:   659+8  
A:   667 
T/F: T
----------
Q:   466+889
A:   1355
T/F: T
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 327us/step - loss: 0.0106 - acc: 0.9999 - val_loss: 0.1295 - val_acc: 0.9570
----------
Q:   54+32  
A:   86  
T/F: T
----------
Q:   753+126
A:   869 
T/F: F
----------
Q:   5+94   
A:   99  
T/F: T
----------
Q:   55+26  
A:   81  
T/F: T
----------
Q:   441+387
A:   818 
T/F: F
----------
Q:   421+138
A:   559 
T/F: T
----------
Q:   123+

18000/18000 [==============================] - 6s 325us/step - loss: 0.0059 - acc: 1.0000 - val_loss: 0.1386 - val_acc: 0.9558
----------
Q:   13+936 
A:   949 
T/F: T
----------
Q:   56+99  
A:   155 
T/F: T
----------
Q:   767+2  
A:   769 
T/F: T
----------
Q:   38+272 
A:   300 
T/F: F
----------
Q:   849+972
A:   1811
T/F: F
----------
Q:   953+42 
A:   995 
T/F: T
----------
Q:   3+277  
A:   280 
T/F: T
----------
Q:   520+58 
A:   568 
T/F: F
----------
Q:   854+80 
A:   924 
T/F: F
----------
Q:   60+58  
A:   118 
T/F: T
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 326us/step - loss: 0.0057 - acc: 1.0000 - val_loss: 0.1396 - val_acc: 0.9550
----------
Q:   617+86 
A:   703 
T/F: T
----------
Q:   221+5  
A:   226 
T/F: T
----------
Q:   977+26 
A:   1003
T/F: T
----------
Q:   4+895  
A:   899 
T/F: T
----------
Q:   766+61 
A:   827 
T/F: T
----------
Q:   3+474  
A:   477 
T/F: T
----------
Q:   223+

18000/18000 [==============================] - 6s 324us/step - loss: 0.0052 - acc: 1.0000 - val_loss: 0.1304 - val_acc: 0.9597
----------
Q:   27+436 
A:   463 
T/F: T
----------
Q:   6+328  
A:   334 
T/F: T
----------
Q:   506+1  
A:   507 
T/F: T
----------
Q:   64+722 
A:   786 
T/F: T
----------
Q:   45+314 
A:   359 
T/F: T
----------
Q:   872+408
A:   1280
T/F: T
----------
Q:   305+44 
A:   358 
T/F: F
----------
Q:   74+63  
A:   137 
T/F: T
----------
Q:   139+6  
A:   145 
T/F: T
----------
Q:   545+626
A:   1171
T/F: T
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 328us/step - loss: 0.0048 - acc: 1.0000 - val_loss: 0.1296 - val_acc: 0.9596
----------
Q:   8+719  
A:   727 
T/F: T
----------
Q:   38+427 
A:   465 
T/F: T
----------
Q:   50+96  
A:   146 
T/F: T
----------
Q:   775+9  
A:   784 
T/F: T
----------
Q:   15+46  
A:   61  
T/F: T
----------
Q:   795+50 
A:   835 
T/F: F
----------
Q:   462+

18000/18000 [==============================] - 6s 337us/step - loss: 0.0028 - acc: 1.0000 - val_loss: 0.1357 - val_acc: 0.9587
----------
Q:   16+569 
A:   584 
T/F: F
----------
Q:   8+826  
A:   834 
T/F: T
----------
Q:   473+65 
A:   538 
T/F: T
----------
Q:   38+829 
A:   867 
T/F: T
----------
Q:   0+531  
A:   531 
T/F: T
----------
Q:   987+78 
A:   1065
T/F: T
----------
Q:   4+844  
A:   848 
T/F: T
----------
Q:   53+44  
A:   97  
T/F: T
----------
Q:   445+69 
A:   514 
T/F: T
----------
Q:   904+18 
A:   922 
T/F: T
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 327us/step - loss: 0.0027 - acc: 1.0000 - val_loss: 0.1366 - val_acc: 0.9580
----------
Q:   3+59   
A:   62  
T/F: T
----------
Q:   767+76 
A:   843 
T/F: T
----------
Q:   57+9   
A:   66  
T/F: T
----------
Q:   198+57 
A:   255 
T/F: T
----------
Q:   882+570
A:   1452
T/F: T
----------
Q:   979+833
A:   1822
T/F: F
----------
Q:   139+